In [1]:
import yfinance as yf
import requests
import pandas as pd
import numpy as np
from functools import reduce
from datetime import timedelta
import plotly.graph_objects as go

In [2]:
top=10
start_period="2020-10-01"
end_period="2022-10-27"
initial_investment = 1000000
n_daysmeasure_perf = 100

In [3]:
def top_stocks(n=50):
    url = "https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%2050"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0;Win64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
    response = requests.get(url, headers=headers)
    return pd.DataFrame(response.json()["data"])['symbol'].iloc[1:n+1]

In [4]:
def stock_history(stock,start_period=None,end_period=None,NS=True):
    temp_df = yf.Ticker(f"{stock}.NS") if NS else yf.Ticker(stock)
    temp_df = temp_df.history(start=start_period,end=end_period)[["Open","Close"]]
    temp_df.columns = map(lambda x: x+"_"+stock,temp_df.columns)
    return temp_df

In [5]:

def calc_equity(temp_df,initial_investment,top):
    
    stock = temp_df.columns[0].split("_")[-1]
    temp_df[f"Daily Value_{stock}"]=temp_df.iloc[:,1]*(initial_investment/top/temp_df.iloc[0,0])
    temp_df.reset_index(inplace=True)
    temp_df['Date']=pd.to_datetime(temp_df['Date'])
    temp_df['Date']=temp_df['Date'].dt.date
    return temp_df

In [6]:
df = reduce(lambda x,y : pd.merge(x,calc_equity(stock_history(y,
                                                              start_period,
                                                              end_period),
                                                initial_investment,
                                                50),
                                  on="Date",
                                  how="outer"),
            top_stocks(),
            pd.DataFrame(columns=["Date"]))

In [7]:
df.shape

(513, 151)

In [8]:
df.sample(3)

,Date,Open_INDUSINDBK,Close_INDUSINDBK,Daily Value_INDUSINDBK,Open_DRREDDY,Close_DRREDDY,Daily Value_DRREDDY,Open_BHARTIARTL,Close_BHARTIARTL,Daily Value_BHARTIARTL,...,Daily Value_HINDALCO,Open_BPCL,Close_BPCL,Daily Value_BPCL,Open_ADANIPORTS,Close_ADANIPORTS,Daily Value_ADANIPORTS,Open_ADANIENT,Close_ADANIENT,Daily Value_ADANIENT
82,2021-01-29,797.836931,826.153259,31337.036550,4780.285958,4499.432129,17617.992046,575.940045,551.150208,25990.608601,...,25541.763327,329.817613,324.488434,21430.166504,520.275595,502.671753,29505.067187,514.639471,506.098755,33799.200062
403,2022-05-19,871.340928,849.998962,32241.534187,3833.817349,3876.683838,15179.556679,681.968787,671.266357,31654.929862,...,47391.066047,324.098221,321.002686,21199.957478,738.881319,727.758362,42716.860926,2144.088463,2113.851318,141171.032176
77,2021-01-21,936.782305,904.023193,34290.741499,4995.350155,4938.847168,19338.567097,597.593856,578.777466,27293.428136,...,28092.550900,351.049794,345.382233,22810.054171,554.102597,540.098450,31701.882895,550.800143,536.315918,35817.217159


In [9]:
df['Equity Curve']=df[[col for col in df.columns if "Daily Value" in col]].sum(axis=1)

/tmp/ipykernel_9383/3792148907.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Equity Curve']=df[[col for col in df.columns if "Daily Value" in col]].sum(axis=1)


In [10]:
df.sample(3)

,Date,Open_INDUSINDBK,Close_INDUSINDBK,Daily Value_INDUSINDBK,Open_DRREDDY,Close_DRREDDY,Daily Value_DRREDDY,Open_BHARTIARTL,Close_BHARTIARTL,Daily Value_BHARTIARTL,...,Open_BPCL,Close_BPCL,Daily Value_BPCL,Open_ADANIPORTS,Close_ADANIPORTS,Daily Value_ADANIPORTS,Open_ADANIENT,Close_ADANIENT,Daily Value_ADANIENT,Equity Curve
469,2022-08-23,1023.768758,1059.427612,40185.427385,4145.000000,4225.149902,16544.011609,725.000000,737.599976,34783.026492,...,330.950012,333.700012,22038.526118,835.799988,832.500000,48864.827374,3032.649902,3030.649902,202398.329136,2.020634e+06
342,2022-02-16,947.975767,950.232666,36043.525159,4220.553433,4258.487305,16674.547657,714.225464,718.605957,33887.324930,...,364.585943,367.632355,24279.517397,719.912758,735.653687,43180.288768,1743.508861,1737.361450,116027.606604,1.828832e+06
165,2021-06-03,1006.694285,982.039551,37250.000471,5190.862632,5164.028320,20220.287198,527.654701,529.695618,24978.873797,...,422.269037,421.735077,27852.619620,807.558849,813.919922,47774.242017,1562.298369,1581.377563,105610.409283,1.616307e+06


In [11]:
# removing daily values as we dont need them
df.drop([col for col in df.columns if "Daily Value" in col],axis=1,inplace=True)

2.

In [12]:
def give_valid(date,df):
    while len(df[df["Date"]==date])!=1:
        date = date-timedelta(1)
    return date

In [13]:
def top_best_per_stock(df,top,n_daysmeasure_perf):
    last_date = give_valid(df["Date"].max()-timedelta(1),df)
    start_date = give_valid(last_date-timedelta(n_daysmeasure_perf),df)
    
    temp_df = df[df["Date"].isin([last_date,start_date])]
    temp_df=temp_df[[i for i in temp_df.columns if "Close" in i]]
    
    temp_df.iloc[0]=temp_df.iloc[0]-1
    temp_df=(temp_df.iloc[1]/(temp_df.iloc[0]-1)).sort_values(ascending=False)
    
    return [i.replace("Close_","") for i in temp_df[:top].index]

In [14]:
top_stocks_names = top_best_per_stock(df,top,n_daysmeasure_perf)

In [15]:
new_df=reduce(lambda x,y : pd.merge(x,calc_equity(df[[col for col in df.columns if y in col]].set_index([df["Date"]]),
                                            initial_investment,
                                            top),
                                  on="Date",
                                  how="outer"),
            top_stocks_names,
            pd.DataFrame(columns=["Date"]))

In [16]:
new_df['Portfolio Equity Curve']=new_df[[col for col in new_df.columns if "Daily" in col]].sum(axis=1)

In [17]:
df=pd.merge(df[["Date","Equity Curve"]],new_df[["Date","Portfolio Equity Curve"]],on="Date")

3.

In [18]:
new_df=calc_equity(stock_history("^NSEI",
                            start_period,
                            end_period,NS=False),
            initial_investment,top=1)

In [19]:
df=pd.merge(df[["Date","Equity Curve","Portfolio Equity Curve"]],new_df[["Date","Daily Value_^NSEI"]],on="Date").rename(columns={"Daily Value_^NSEI":"Nifty Index Equity Curve"})

4.

In [20]:
df

,Date,Equity Curve,Portfolio Equity Curve,Nifty Index Equity Curve
0,2020-10-01,1.005403e+06,1.026593e+06,1.004620e+06
1,2020-10-05,1.012927e+06,1.029134e+06,1.012222e+06
2,2020-10-06,1.022582e+06,1.041954e+06,1.026218e+06
3,2020-10-07,1.023996e+06,1.039530e+06,1.032945e+06
4,2020-10-08,1.029565e+06,1.035949e+06,1.041370e+06
...,...,...,...,...
507,2022-10-19,2.027593e+06,3.163749e+06,1.540968e+06
508,2022-10-20,2.042168e+06,3.188422e+06,1.545517e+06
509,2022-10-21,2.037438e+06,3.178222e+06,1.546604e+06
510,2022-10-24,2.053037e+06,3.196542e+06,1.560194e+06


In [21]:
# Calculating the number of years
t=(pd.to_datetime(end_period) - pd.to_datetime(start_period)) / timedelta(365)

In [22]:
# def CAGR(df,t):
#     return ((df.iloc[-1]/df.iloc[0])**(1/t)-1)*100

# def daily_return(df):
#     return ((df.iloc[-1]/df.iloc[0])**(1/t)-1)*100

# def Volatility(df):
#     return (daily_return(df).std()**(1/252))*100

# def Sharpe_Ratio(df):
#     d_r = daily_return(df)
#     return (d_r.mean()/d_r.std())**(1/252)
    
    
def performance(df,t):
    """_summary_

    Args:
        df (pd.Series): column
        t (float/int): year

    Returns:
        tupple: CAGR (%), Volatility (%), Sharpe Ratio
    """
    d_r = ((df/df.shift(1))-1).dropna()     # daily return
    std_d_r = np.std(d_r)
    
    return ((df.iloc[-1]/df.iloc[0])**(1/t)-1)*100, (std_d_r**(1/252))*100, (np.mean(d_r)/std_d_r)**(1/252)

In [23]:
final_df=pd.DataFrame({"Equally Alloc Buy Hold":performance(df['Equity Curve'],t),
                        "Nifty":performance(df['Nifty Index Equity Curve'],t),
                        "Performance_Strat":performance(df['Portfolio Equity Curve'],t)},
                      index=["CAGR (%)", "Volatility (%)", "Sharpe Ratio"])

In [24]:
final_df=final_df.T

In [25]:
final_df

,CAGR (%),Volatility (%),Sharpe Ratio
Equally Alloc Buy Hold,41.081714,98.236695,0.991913
Nifty,23.429615,98.209765,0.990323
Performance_Strat,72.610379,98.348073,0.992626


In [26]:
df

,Date,Equity Curve,Portfolio Equity Curve,Nifty Index Equity Curve
0,2020-10-01,1.005403e+06,1.026593e+06,1.004620e+06
1,2020-10-05,1.012927e+06,1.029134e+06,1.012222e+06
2,2020-10-06,1.022582e+06,1.041954e+06,1.026218e+06
3,2020-10-07,1.023996e+06,1.039530e+06,1.032945e+06
4,2020-10-08,1.029565e+06,1.035949e+06,1.041370e+06
...,...,...,...,...
507,2022-10-19,2.027593e+06,3.163749e+06,1.540968e+06
508,2022-10-20,2.042168e+06,3.188422e+06,1.545517e+06
509,2022-10-21,2.037438e+06,3.178222e+06,1.546604e+06
510,2022-10-24,2.053037e+06,3.196542e+06,1.560194e+06


In [32]:
fig=go.Figure()

temp_store = []

fig.add_scatter(x=df['Date'],y=df["Equity Curve"],name="Equally Alloc Buy Hold",marker=dict(color="red"))
fig.add_scatter(x=df['Date'],y=df["Nifty Index Equity Curve"],name="Nifty",marker=dict(color="blue"))
fig.add_scatter(x=df['Date'],y=df["Portfolio Equity Curve"],name="Performance_Strat",marker=dict(color="green"))
fig.update_layout(hovermode="x unified")
fig.show()